In [1]:
import os
import glob 
import pandas as pd
import numpy as np
os.chdir(r"C:\Users\ehud\Documents\bell-pepper\2025-04-0609\control") # Set the current working directory
csv_files=glob.glob("*.csv")  # Find all .csv files in the current working directory

In [2]:
# from each csv file in the current working directory, read only the "Intensity" column and concatenate all columns to one dataframe, which we shall call `dataset`:
dataset=pd.concat(
    [
        pd.read_csv(file, usecols=["Intensity"]).rename(
            columns={"Intensity": os.path.splitext(file)[0]}
        )
        for file in csv_files
    ],
    axis=1
)

In [3]:
# prepend to this dataframe the wavelength column and reset the result. The wv column is taken from one of the csv files that has already been read to the above dataframe.
# All wv columns in this directory should be identical; For convenience we take the column from the first file.

# Read the Wavelength column from the first csv file
wavelength = pd.read_csv(csv_files[0], usecols=["Wavelength"])


dataset=pd.concat([pd.read_csv(csv_files[0], usecols=["Wavelength"]).rename(columns={"Wavelength":"wv"}), dataset], axis=1)

In [ ]:
#data.to_csv("output.csv",index=False)

In [4]:
dataset

,wv,001,002,003,004,005,006,007,008,009,...,115,116,117,118,119,120,15_24_51,15_39_01,16_23_30,17_13_28
0,804.404322,45039,42307,43484,43332,43393,42074,40452,39089,40658,...,48983,46583,39712,42611,41616,46040,46415,32335,40752,37991
1,804.674093,45915,42891,43971,43947,43647,42417,40498,39754,41638,...,49244,47056,39904,42710,42243,46668,46664,32680,41031,38378
2,804.943872,45839,42942,44293,44291,44051,42710,40666,40083,41605,...,49797,47138,40215,43015,42405,47045,47125,32995,41510,38722
3,805.213661,45979,42877,44104,44460,44192,42296,40978,40060,41909,...,50502,47634,40446,43151,42419,47658,47303,32976,41862,39123
4,805.483458,46018,42770,44321,44629,43792,42326,41035,40250,42079,...,50233,47799,40582,42790,43188,47391,47299,33048,41484,39010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,1080.210034,6522,5097,5336,3817,4928,7308,5225,4432,3307,...,4656,3308,5417,9102,2287,5499,5835,3155,6954,8028
1020,1080.477879,6343,5038,5368,3771,4961,7159,5276,4399,3209,...,4610,3219,5346,9125,2260,5388,5686,3140,6804,7922
1021,1080.745711,6313,4950,5351,3720,4766,7031,5207,4441,3241,...,4521,3195,5338,8863,2314,5394,5589,3056,6762,7780
1022,1081.013530,6337,5021,5174,3713,4814,7140,5199,4434,3168,...,4435,3295,5332,8836,2176,5356,5529,2996,6825,7876


In [19]:
# Create a new dataframe with the value of numerical integral of spectral intensity vs. wavelength, for each column:
results = {}

for col in range(dataset.shape[1] - 1):
    col_name = dataset.columns[col + 1]
    integral = np.trapezoid(dataset.iloc[:, col + 1], dataset['wv'])
    results[col_name] = integral
# Convert the results to a DataFrame if needed
integrals = pd.DataFrame([results])
del results
del col_name
del integral

In [20]:
integrals

,001,002,003,004,005,006,007,008,009,010,...,115,116,117,118,119,120,15_24_51,15_39_01,16_23_30,17_13_28
0,7.724250e+06,7.381679e+06,7.092889e+06,6.912469e+06,7.082445e+06,7.149810e+06,6.552709e+06,6.152600e+06,6.390294e+06,6.738755e+06,...,8.035396e+06,7.445402e+06,6.333469e+06,7.145666e+06,6.229520e+06,7.382522e+06,7.573250e+06,4.891542e+06,6.712209e+06,6.737056e+06


In [ ]:
# check that `openpyxl` is installed (pandas engine used to read xls files.
# pip install openpyxl
# read the correspondence file that matches measurement and its exposure time
deltat=pd.read_excel(r"../correspondence_exposure_times.xlsx",sheet_name="Control").iloc[:, -2:]


,file name,exposure time (s)
0,1.0,7.0
1,2.0,5.5
2,3.0,5.5
3,4.0,3.5
4,5.0,5.0
...,...,...
116,117.0,5.5
117,118.0,10.0
118,119.0,1.7
119,120.0,5.0


In [ ]:
# Append a new row to the integrals dataset using values from `deltat`'s 'time' column, matched by column names in `integrals`` and filenames in deltat.

integrals_df.loc[1]= deltat.set_index('filename')['time'].reindex(integrals_df.columns)
print(integrals_df)
#print(df2)

KeyError: "None of ['filename'] are in the columns"